In [1]:
import dash                                     # pip install dash
from dash import dcc, html, Output, Input
import plotly.express as px
import dash_bootstrap_components as dbc
import pandas as pd
from plotly.subplots import make_subplots

In [2]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX])
df = pd.read_csv(r'C:\Users\syzmon\Desktop\IBM data science capstone\db.csv')
df.head()
df4stat = df.groupby(['CONTINENT', 'YEAR'])['ID'].count().reset_index()
df4stat.rename(columns={'ID':'COUNT'}, inplace=True)
ds_min = df4stat['YEAR'].min()
ds_max = df4stat['YEAR'].max()
ds_median = df4stat.groupby(['YEAR']).median()
marks={
        i: {
            "label": f"{i}",
            "style": {"transform": "rotate(0deg)", "white-space": "nowrap"},
        }
        for i in range(ds_min, ds_max, 10)
}

C:\Users\syzmon\AppData\Local\Temp\ipykernel_5212\540845490.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ds_median = df4stat.groupby(['YEAR']).median()


In [3]:
app.layout = html.Div([
    html.H1('Meteorite Landings from NASA dataset',
        style={'textAlign': 'center','color':'#503D36','font-size':75}
    ),
    html.Div([
        html.Div([
            dcc.Checklist(id='my-checklist', value=df['CONTINENT'].unique(),
                            options=[{'label': i, 'value': i} for i in sorted(df['CONTINENT'].unique())],
                            style={'font-size': '20px'},
                            labelClassName='mr-3',
                            inline=True
                        )
            ],style={'width':'50%', "margin": "15px"}),

        html.Div([
            html.Label("Select Year", htmlFor="years", style={'font-size': '20px', 'width':'50%'}),
            dcc.RangeSlider(id='sslider',
                            min=ds_min, max=ds_max, step=10, 
                            value=[df4stat['YEAR'].min(), df4stat['YEAR'].max()],
                            updatemode='drag',
                            tooltip={"placement": 'bottom', "always_visible": True},
                            marks=marks, 
                        ),
            ], style={'width':'50%'})
            
    ],style={'display':'flex'}),
    
    html.Div([
        dcc.Graph(id='my-hist', figure={}, style={'width':'50%'}),
        dcc.Graph(id="graph", figure={}, style={'width':'50%'})
    ],style={'display':'flex'}),
    
    dcc.Dropdown(id='dpdn2', value=['North America','Europe', 'Asia'], multi=True,
                 options=[{'label': x, 'value': x} for x in
                          df['CONTINENT'].unique()]),
    html.Div([
        dcc.Graph(id='pie-graph', figure={},style={'width':'50%'}, className='six columns'),
        dcc.Graph(id='my-graph', figure={}, clickData=None, hoverData=None, # I assigned None for tutorial purposes. By defualt, these are None, unless you specify otherwise.
                  config={
                      'staticPlot': False,     # False - Graph is static, so you can't for example save image
                      'scrollZoom': True,      # True - Allows you to zoom your graph
                      'doubleClick': 'reset',  # 'reset' - Reset image zoom on double click 
                      'showTips': False,       # Tips for users
                      'displayModeBar': True,  # Mode bar
                      'watermark': True,       # plotly watermark 
                      'modeBarButtonsToRemove': ['select2d'] # Remove mode bar features
                    },style={'width':'50%'},
                  className='six columns'
                  )
    ], style={'display':'flex'})
])

In [4]:
@app.callback(
    Output(component_id='my-hist', component_property='figure'),
    [Input(component_id='my-checklist', component_property='value')]
)
def update_hist(continents):
    df_hist = df.groupby(['CONTINENT'])['ID'].count().reset_index()
    df_hist = df_hist[df_hist['CONTINENT'].isin(continents)]
    print(continents)
    fig = px.bar(x=df_hist['CONTINENT'], y=df_hist['ID'], labels={'x':'Continent', 'y':'Count'})
    return fig


In [5]:
@app.callback(
    Output(component_id='graph', component_property='figure'),
    [Input(component_id='sslider', component_property='value')]
)
def update_graph(years):
    dff = df[df['YEAR']>=years[0]]
    dff = dff[dff['YEAR']<=years[1]]
    dff = dff.groupby(['CONTINENT', 'YEAR'])['ID'].count().reset_index()


    dff = dff.pivot(index='CONTINENT', columns='YEAR')['ID'].fillna(0)
    fig=px.imshow(dff, x=dff.columns , y=dff.index)
    fig.update_layout(title=f"Heat map for years: {years}", xaxis_title='year', yaxis_title='cont')
    return (fig)

In [6]:
@app.callback(
    Output(component_id='my-graph', component_property='figure'),
    Input(component_id='dpdn2', component_property='value'),
)
def update_graph(continent_chosen):
    dff = df4stat[df4stat.CONTINENT.isin(continent_chosen)]
    fig = px.scatter(data_frame=dff, x='YEAR', y='COUNT', color='CONTINENT')
    return fig

In [7]:
# Dash version 1.16.0 or higher
@app.callback(
    Output(component_id='pie-graph', component_property='figure'),
    Input(component_id='my-graph', component_property='hoverData'),
    Input(component_id='my-graph', component_property='clickData'),
    Input(component_id='my-graph', component_property='selectedData'),
    Input(component_id='dpdn2', component_property='value')
)
def update_side_graph(hov_data, clk_data, slct_data, continent_chosen):
    if hov_data is None:
        dff2 = df4stat[df4stat['YEAR'] == ds_min]
        dff2 = df4stat[df4stat.CONTINENT.isin(continent_chosen)]
        fig2 = px.pie(data_frame=dff2, values='COUNT', names='CONTINENT',
                      title=f'Year: {ds_min}')
        return fig2
    else:
        dff2 = df4stat[df4stat.CONTINENT.isin(continent_chosen)]
        hov_year = hov_data['points'][0]['x']
        dff2 = dff2[dff2['YEAR'] == hov_year]
        fig2 = px.pie(data_frame=dff2, values='COUNT', names='CONTINENT', title=f'Year: {hov_year}')
        return fig2


In [8]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [03/Jun/2023 12:23:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:19] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:19] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:19] "GET /_favicon.ico?v=2.9.3 HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:19] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Jun/2023 12:23:19] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Jun/2023 12:23:20] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -


['North America', 'Asia', 'Africa', 'Europe', 'South America', 'Antarctica', 'Oceania', 'Nan']


127.0.0.1 - - [03/Jun/2023 12:23:20] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [03/Jun/2023 12:23:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:26] "POST /_dash-update-component HTTP/1.1" 200 -


['North America', 'Asia', 'Africa', 'Europe', 'Antarctica', 'Oceania', 'Nan']


...

127.0.0.1 - - [03/Jun/2023 12:23:28] "POST /_dash-update-component HTTP/1.1" 200 -


['Asia', 'Africa', 'Europe', 'Antarctica', 'Nan']


127.0.0.1 - - [03/Jun/2023 12:23:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:34] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:23:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.

['North America', 'Asia', 'Africa', 'Europe', 'South America', 'Antarctica', 'Oceania', 'Nan']


127.0.0.1 - - [03/Jun/2023 12:25:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:25:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:25:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:25:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:25:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:25:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:25:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:25:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:25:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:25:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:25:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2023 12:25:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.